# Model Deployment Class, replicating

In [1]:
from helper_functions import accuracy_fn, download_data, plot_decision_boundary, plot_loss_curves, plot_predictions, set_seeds
# from helper_functions import download_data, set_seeds, plot_loss_curves
from going_modular import engine, data_loaders
from going_modular import predictions
from going_modular.engine import train_step, test_step
from going_modular.utils import save_model
import mlxtend
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
import os
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import random
import shutil
import requests
import sklearn
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from torchinfo import summary
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import Accuracy, ConfusionMatrix
import torchvision
from torchvision import datasets

from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import Tuple, Dict, List
writer = SummaryWriter()
import zipfile



c:\Users\Tiger\anaconda3\envs\Py311UdemyWCUDA1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# see torch and torch vision plus
print (torch.__version__)
print (torchvision.__version__)

2.3.0
0.18.0


In [3]:
# !nvcc --version

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def set_seeds (seed: int=42):
    """_summary_

    Args:
        seed (int, optional): _description_. Defaults to 42.
    """ 
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)   

In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
# wet up data paths
# E:\Dropbox\GithubRepo\Udemy\pytorch-deep-learning-main\pytorch-deep-learning-main - Copy\data\pizza_steak_sushi_20_percent
data_20_percent_path = Path('../data/pizza_steak_sushi_20_percent')
train_dir = data_20_percent_path / 'train'
test_dir = data_20_percent_path / 'test'
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2_transforms = effnetb2_weights.transforms()
print (f'effnetb2 transforms: {effnetb2_transforms}')
# effnetb2 = torchvision.models.efficientnet_b2(weights="effnetb2_weights")
effnetb2 = torchvision.models.efficientnet_b2(weights="DEFAULT")

for param in effnetb2.parameters():
    param.requires_grad=False


effnetb2 transforms: ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


In [7]:
from torchinfo import summary
# effnetb2
summary(model=effnetb2,
        input_size= (1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 1000]            --                   False
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16,

In [8]:
# get input size of last layer
effnetb2_lastlayer = list(effnetb2.children())[-1]
effnetb2_last_child_layer_inpu_count = list(effnetb2_lastlayer.children())[-1].in_features
effnetb2_last_child_layer_inpu_count

1408

In [9]:
set_seeds()
effnetb2.classifier=nn.Sequential(
    nn.Dropout(p = 0.3, inplace=True),
    nn.Linear(in_features=effnetb2_last_child_layer_inpu_count, out_features=3)

)

In [10]:
# function to create effnetb2
def create_effnetb2(output_feature_count:int, drop_out = 0.3, seed:int=42)-> nn.Module:
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms= weights.transforms()
    # model = torchvision.models.efficientnet_b2(weights="DEFAULT").to(device)
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    last_layer = list(model.children())[-1]
    last_child_layer = list(last_layer.children())[-1]
    last_input = last_child_layer.in_features

    model.classifier = nn.Sequential(
        nn.Dropout(p = drop_out, inplace=True),
        nn.Linear(in_features =last_input, 
                  out_features = output_feature_count).to(device))

    model.name = 'effnetb2'
    print (f'[INFO] created new {model.name} model')
    
    return model, transforms




In [11]:
effnetb2, effnetb2_transforms = create_effnetb2(output_feature_count=3)
effnetb2
print(summary(model=effnetb2,
        input_size= (1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names']))

effnetb2_transforms

[INFO] created new effnetb2 model
Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                

ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [12]:
# data loader
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = data_loaders.create_dataloaders(train_dir=train_dir, 
                                                                                          test_dir=test_dir, # use 10% data for testing
                                                                                          transform=effnetb2_transforms, 
                                                                                          batch_size=32)

In [13]:
len (train_dataloader_effnetb2), len (test_dataloader_effnetb2), class_names

(15, 5, ['pizza', 'steak', 'sushi'])

In [14]:
loss_fn = nn.CrossEntropyLoss()
effnetb2_optimizer = torch.optim.Adam(effnetb2.parameters(), lr=1e-3)

In [15]:
# general trainer

def train (model: torch.nn.Module,
           train_dataloader: torch.utils.data.DataLoader,
           test_dataloader: torch.utils.data.DataLoader,
           optimizer: torch.optim.Optimizer,
           loss_fn: torch.nn.Module,
           epochs: int,
           device: torch.device,
           writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:
    
    results = {'train_loss':[],
               'train_acc': [],
               'test_loss': [],
               'test_acc': []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model = model,
                                           dataloader = train_dataloader,
                                           loss_fn = loss_fn,
                                           optimizer=optimizer,
                                           device = device)
        
        test_loss, test_acc = test_step(model = model,
                                        dataloader = test_dataloader,
                                        loss_fn = loss_fn,
                                        device=device)
        
        print (
            f'Epoch: {epoch + 1} |'
            f'train_loss: {train_loss:.4f} |'
            f'train_acc: {train_acc:.4f} |'
            f'test_loss {test_loss:.4f} |'
            f'test_acc:  {test_acc}:.4f'

        )

        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    if writer:
        writer.add_scalars (main_tag = 'Accuracy',
                            tag_scalar_dict = {'train_acc': train_acc,
                                              'test_acc':test_acc},
                            global_step=epoch)
        writer.add_scalars (main_tag = 'Loss',
                           tag_scalar_dict = {'train_loss': train_loss,
                                              'test_loss' : test_loss},
                            global_step = epoch)
        
        writer.add_graph (model = model,
                          input_to_model = torch.randn(32, 3, 224, 224).to(device))# 3 or 1?
    
    writer.close()

    return results

In [16]:
# train effnetb2
set_seeds()

effnetb2_retults = train (model = effnetb2,
                          train_dataloader=train_dataloader_effnetb2,
                          test_dataloader=test_dataloader_effnetb2,
                          optimizer=effnetb2_optimizer,
                          loss_fn = loss_fn,
                          epochs = 10,
                          device=device,
                          writer=writer)

  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\Tiger\anaconda3\envs\Py311UdemyWCUDA1\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
 10%|█         | 1/10 [00:05<00:49,  5.54s/it]

Epoch: 1 |train_loss: 0.9855 |train_acc: 0.5604 |test_loss 0.7407 |test_acc:  0.934659090909091:.4f


 20%|██        | 2/10 [00:12<00:49,  6.18s/it]

Epoch: 2 |train_loss: 0.7175 |train_acc: 0.8438 |test_loss 0.5869 |test_acc:  0.940909090909091:.4f


 30%|███       | 3/10 [00:18<00:43,  6.27s/it]

Epoch: 3 |train_loss: 0.5876 |train_acc: 0.8917 |test_loss 0.4909 |test_acc:  0.95:.4f


 40%|████      | 4/10 [00:23<00:35,  5.84s/it]

Epoch: 4 |train_loss: 0.4474 |train_acc: 0.9062 |test_loss 0.4355 |test_acc:  0.940909090909091:.4f


 50%|█████     | 5/10 [00:29<00:28,  5.77s/it]

Epoch: 5 |train_loss: 0.4290 |train_acc: 0.9104 |test_loss 0.3915 |test_acc:  0.9443181818181818:.4f


 60%|██████    | 6/10 [00:35<00:23,  5.75s/it]

Epoch: 6 |train_loss: 0.4380 |train_acc: 0.8896 |test_loss 0.3512 |test_acc:  0.96875:.4f


 70%|███████   | 7/10 [00:41<00:17,  5.99s/it]

Epoch: 7 |train_loss: 0.4245 |train_acc: 0.8771 |test_loss 0.3268 |test_acc:  0.95625:.4f


 80%|████████  | 8/10 [00:49<00:13,  6.70s/it]

Epoch: 8 |train_loss: 0.3897 |train_acc: 0.8958 |test_loss 0.3457 |test_acc:  0.9289772727272727:.4f


 90%|█████████ | 9/10 [00:55<00:06,  6.52s/it]

Epoch: 9 |train_loss: 0.3749 |train_acc: 0.8812 |test_loss 0.3129 |test_acc:  0.9130681818181818:.4f


100%|██████████| 10/10 [01:01<00:00,  6.16s/it]

Epoch: 10 |train_loss: 0.3757 |train_acc: 0.8604 |test_loss 0.2813 |test_acc:  0.96875:.4f


In [17]:
# !nvidia-smi
plot_loss_curves(effnetb2_retults)

### 3.5 Save the EffNetB2 feeature extractor

In [18]:
model_path = f"09_pre_trained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent_{10}_epochs.pth"
save_model (model = effnetb2,
            target_dir="models",
            model_name = model_path)

[INFO] Saving model to: models\09_pre_trained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent_10_epochs.pth


### 3.6 inspect size of EFFNetB2

In [19]:
pretrained_effnetb2_model_size = Path("models/", model_path).stat().st_size/(1024 * 1024)

### 3.7 Colling EffNetB2 feature extractor stats

In [20]:
# count number of paramters
effnetb2_total_params = sum(torch.numel(param) for param in effnetb2.parameters())
effnetb2_total_params

7705221

In [21]:
effnetb2_stats = {'test_loss':effnetb2_retults['test_loss'][-1],
                  'test_acc':effnetb2_retults['test_acc'][-1],
                  'number_of_paramters': effnetb2_total_params,
                  'model_size (MB)': pretrained_effnetb2_model_size}

effnetb2_stats

{'test_loss': 0.2812606513500214,
 'test_acc': 0.96875,
 'number_of_paramters': 7705221,
 'model_size (MB)': 29.89214515686035}

## 4.0 ViT feature extractor

In [22]:
vit = torchvision.models.vit_b_16()
vit.heads

Sequential(
  (head): Linear(in_features=768, out_features=1000, bias=True)
)

In [23]:
# list(list(vit.children())[-1])[-1].in_features

In [24]:
# create vit model
def create_vit_model(num_classes: int = 3, drop_out = 0.3, seed:int=42)-> nn.Module:
    weights = torchvision.models.ViT_B_16_Weights.DEFAULT
    model= torchvision.models.vit_b_16(weights=weights).to(device)
    transforms=weights.transforms()

    for param in model.parameters():
        param.requires_grad = False

    set_seeds(seed=seed)

    last_layer = list (model.children())[-1]
    last_child_layer = list (last_layer.children())[-1]
    last_input = last_child_layer.in_features

    model.heads = nn.Sequential(
        nn.Dropout(p = drop_out, inplace=True),
        nn.Linear(in_features=last_input,
                  out_features=num_classes).to(device))
    
    model.name = 'ViT_B_16'
    print (f'[INFO] created new {model.name} model')

    return model, transforms

In [25]:
vit, vit_transforms = create_vit_model(num_classes=3)

print(summary(model=vit,
        input_size= (1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names']))

vit_transforms

[INFO] created new ViT_B_16 model
Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [1, 3, 224, 224]     [1, 3]               768                  Partial
├─Conv2d (conv_proj)                                         [1, 3, 224, 224]     [1, 768, 14, 14]     (590,592)            False
├─Encoder (encoder)                                          [1, 197, 768]        [1, 197, 768]        151,296              False
│    └─Dropout (dropout)                                     [1, 197, 768]        [1, 197, 768]        --                   --
│    └─Sequential (layers)                                   [1, 197, 768]        [1, 197, 768]        --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [1, 197, 768]        [1, 197, 768]        (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)           

c:\Users\Tiger\anaconda3\envs\Py311UdemyWCUDA1\Lib\site-packages\torch\nn\modules\activation.py:1221: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  return torch._native_multi_head_attention(


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [26]:
effnetb2, effnetb2_transforms = create_effnetb2(output_feature_count=3)
effnetb2
print(summary(model=effnetb2,
        input_size= (1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names']))

effnetb2_transforms

[INFO] created new effnetb2 model
Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                

ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

### 4.1 ViT Dataloader

In [27]:
# data loader
train_dataloader_vit, test_dataloader_vit, class_names = data_loaders.create_dataloaders(train_dir=train_dir, 
                                                                                          test_dir=test_dir, # use 10% data for testing
                                                                                          transform=vit_transforms, 
                                                                                          batch_size=32)

### 4.2 Vit training

In [28]:
loss_fn = nn.CrossEntropyLoss()
vit_optimizer = torch.optim.Adam(vit.parameters(), lr=1e-3)

In [29]:
# train vit
set_seeds()

vit_retults = train (model = vit,
                          train_dataloader=train_dataloader_vit,
                          test_dataloader=test_dataloader_vit,
                          optimizer=vit_optimizer,
                          loss_fn = loss_fn,
                          epochs = 10,
                          device=device,
                          writer=writer)

 10%|█         | 1/10 [00:06<00:57,  6.44s/it]

Epoch: 1 |train_loss: 0.7245 |train_acc: 0.7250 |test_loss 0.2896 |test_acc:  0.9289772727272727:.4f


 20%|██        | 2/10 [00:12<00:51,  6.39s/it]

Epoch: 2 |train_loss: 0.2648 |train_acc: 0.9083 |test_loss 0.1720 |test_acc:  0.9602272727272727:.4f


 30%|███       | 3/10 [00:18<00:43,  6.28s/it]

Epoch: 3 |train_loss: 0.1975 |train_acc: 0.9479 |test_loss 0.1281 |test_acc:  0.9693181818181819:.4f


 40%|████      | 4/10 [00:25<00:37,  6.26s/it]

Epoch: 4 |train_loss: 0.1410 |train_acc: 0.9604 |test_loss 0.1085 |test_acc:  0.972159090909091:.4f


 50%|█████     | 5/10 [00:31<00:30,  6.15s/it]

Epoch: 5 |train_loss: 0.1234 |train_acc: 0.9646 |test_loss 0.0965 |test_acc:  0.978409090909091:.4f


 60%|██████    | 6/10 [00:36<00:24,  6.03s/it]

Epoch: 6 |train_loss: 0.1387 |train_acc: 0.9354 |test_loss 0.0846 |test_acc:  0.972159090909091:.4f


 70%|███████   | 7/10 [00:42<00:17,  5.98s/it]

Epoch: 7 |train_loss: 0.1079 |train_acc: 0.9625 |test_loss 0.0831 |test_acc:  0.9875:.4f


 80%|████████  | 8/10 [00:48<00:11,  5.92s/it]

Epoch: 8 |train_loss: 0.1237 |train_acc: 0.9437 |test_loss 0.0761 |test_acc:  0.978409090909091:.4f


 90%|█████████ | 9/10 [00:54<00:05,  5.90s/it]

Epoch: 9 |train_loss: 0.1052 |train_acc: 0.9771 |test_loss 0.0750 |test_acc:  0.978409090909091:.4f


100%|██████████| 10/10 [01:00<00:00,  6.02s/it]

Epoch: 10 |train_loss: 0.0786 |train_acc: 0.9854 |test_loss 0.0640 |test_acc:  1.0:.4f



c:\Users\Tiger\anaconda3\envs\Py311UdemyWCUDA1\Lib\site-packages\torch\__init__.py:1559: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


In [30]:
# train effnetb2
plot_loss_curves(vit_retults)

### 4.3 Save the fit feeature extractor

In [31]:
vit_model_path = f"09_pre_trained_vit_feature_extractor_pizza_steak_sushi_20_percent_{10}_epochs.pth"
save_model (model = vit,
            target_dir="models",
            model_name = vit_model_path)

[INFO] Saving model to: models\09_pre_trained_vit_feature_extractor_pizza_steak_sushi_20_percent_10_epochs.pth


### 4.4 inspect size of vit

In [32]:
pretrained_vit_model_size = Path("models/", vit_model_path).stat().st_size/(1024 * 1024)
pretrained_vit_model_size

327.38194465637207

### 4.6 Collecting vit feature extractor stats

In [33]:
# count number of paramters
vit_total_params = sum(torch.numel(param) for param in vit.parameters())
vit_total_params

85800963

In [34]:
vit_stats = {'test_loss':vit_retults['test_loss'][-1],
                  'test_acc':vit_retults['test_acc'][-1],
                  'number_of_paramters': vit_total_params,
                  'model_size (MB)': pretrained_vit_model_size}

vit_stats

{'test_loss': 0.06396642997860909,
 'test_acc': 1.0,
 'number_of_paramters': 85800963,
 'model_size (MB)': 327.38194465637207}

## 5. Making Prediction swith our trained data models


In [35]:
# Get all test data paths
# from tqdm import tqdm
# from pathlib import Path
# test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
# test_labels = [path.parent.stem for path in test_data_paths]
# Create a function to return a list of dictionaries with sample, label, prediction, pred prob
def pred_and_store(test_paths, model, transform, class_names, device):
  test_pred_list = []
  for path in tqdm(test_paths):
    # Create empty dict to store info for each sample
    pred_dict = {}

    # Get sample path
    pred_dict["image_path"] = path

    # Get class name
    class_name = path.parent.stem
    pred_dict["class_name"] = class_name

    #520cc050-f648-4d4d-b7ae-16850d1a8fbd Get prediction and prediction probability
    from PIL import Image
    
    # 6. timer
    start_time = timer()

    img = Image.open(path) # open image
    transformed_image = transform(img).unsqueeze(0) # transform image and add batch dimension
    model.eval()
    with torch.inference_mode():
      pred_logit = model(transformed_image.to(device))
      pred_prob = torch.softmax(pred_logit, dim=1)
      pred_label = torch.argmax(pred_prob, dim=1)
      # pred_class = class_names[pred_label.cpu()]
      pred_class = class_names[pred_label.cpu()]

      # Make sure things in the dictionary are back on the CPU 
      pred_dict["pred_prob"] = pred_prob.unsqueeze(0).max().cpu().item()
      pred_dict["pred_class"] = pred_class

      # 12. End timerer
      end_time = timer()
      pred_dict["time_for_pred"] = round (end_time - start_time, 4)
  
    # Does the pred match the true label?
    pred_dict["correct"] = class_name == pred_class

    # print(pred_dict)
    # Add the dictionary to the list of preds
    test_pred_list.append(pred_dict)

  return test_pred_list

# test_pred_dicts = pred_and_store(test_paths=test_data_paths,
#                                  model=pretrained_vit_swag,
#                                  transform=vit_transforms_swag,
#                                  class_names=class_names,
#                                  device=device)

# test_pred_dicts[:5]

In [36]:
# test on vit
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
test_labels = [path.parent.stem for path in test_data_paths]
test_pred_dicts_vit = pred_and_store(test_paths=test_data_paths,
                                 model=vit,
                                 transform=vit_transforms,
                                 class_names=class_names,
                                 device=device)

test_pred_dicts_vit[:5]

100%|██████████| 150/150 [00:02<00:00, 62.62it/s]


[{'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1001116.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9985803365707397,
  'pred_class': 'pizza',
  'time_for_pred': 0.0215,
  'correct': True},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1032754.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9952054023742676,
  'pred_class': 'pizza',
  'time_for_pred': 0.0142,
  'correct': True},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1067986.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9984466433525085,
  'pred_class': 'pizza',
  'time_for_pred': 0.0164,
  'correct': True},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/129666.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9858282804489136,
  'pred_class': 'pizza',
  'time_for_pred': 0.0162,
  'correct': True},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg'),
  'class_name

In [37]:
# 
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
test_labels = [path.parent.stem for path in test_data_paths]
test_pred_dicts_effnetb2 = pred_and_store(test_paths=test_data_paths,
                                 model=effnetb2,
                                 transform=effnetb2_transforms,
                                 class_names=class_names,
                                 device=device)

test_pred_dicts_effnetb2[:5]

100%|██████████| 150/150 [00:04<00:00, 35.58it/s]


[{'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1001116.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.40349331498146057,
  'pred_class': 'sushi',
  'time_for_pred': 0.1072,
  'correct': False},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1032754.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.3899497091770172,
  'pred_class': 'sushi',
  'time_for_pred': 0.0313,
  'correct': False},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1067986.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.36351701617240906,
  'pred_class': 'sushi',
  'time_for_pred': 0.0281,
  'correct': False},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/129666.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.36818185448646545,
  'pred_class': 'sushi',
  'time_for_pred': 0.0279,
  'correct': False},
 {'image_path': WindowsPath('../data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg'),
  'cla

In [38]:
effnet2b_test_pred_df = pd. DataFrame(test_pred_dicts_effnetb2)
effnet2b_test_pred_df.head(2)

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,..\data\pizza_steak_sushi_20_percent\test\pizz...,pizza,0.403493,sushi,0.1072,False
1,..\data\pizza_steak_sushi_20_percent\test\pizz...,pizza,0.389950,sushi,0.0313,False


In [39]:
# check number of correct predictions
effnet2b_test_pred_df.correct.value_counts()

correct
False    117
True      33
Name: count, dtype: int64

In [40]:
# Find the average time per prediction
effnetb2_average_time_per_pred = round(effnet2b_test_pred_df.time_for_pred.mean(), 4)
print(f"EffNetB2 average time per prediction: {effnetb2_average_time_per_pred} seconds")

EffNetB2 average time per prediction: 0.0279 seconds


In [41]:
effnetb2_stats['time_per_pred_cpu'] =effnetb2_average_time_per_pred
effnetb2_stats

{'test_loss': 0.2812606513500214,
 'test_acc': 0.96875,
 'number_of_paramters': 7705221,
 'model_size (MB)': 29.89214515686035,
 'time_per_pred_cpu': 0.0279}

###  5.3 get statsfor stats VIT

In [42]:
# create DF
vit_test_pred_df = pd.DataFrame(test_pred_dicts_vit)
vit_test_pred_df.head(2)

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,..\data\pizza_steak_sushi_20_percent\test\pizz...,pizza,0.998580,pizza,0.0215,True
1,..\data\pizza_steak_sushi_20_percent\test\pizz...,pizza,0.995205,pizza,0.0142,True


In [43]:
# Count the number of correct predictions
vit_test_pred_df.correct.value_counts()

correct
True    150
Name: count, dtype: int64

In [44]:
# get average
vit_average_time_per_pred = round (vit_test_pred_df.time_for_pred.mean(), 4)
vit_average_time_per_pred

0.0158

In [45]:
vit_stats['time_per_pred_cpu'] = vit_average_time_per_pred
vit_stats

{'test_loss': 0.06396642997860909,
 'test_acc': 1.0,
 'number_of_paramters': 85800963,
 'model_size (MB)': 327.38194465637207,
 'time_per_pred_cpu': 0.0158}

## 6. Comparing models 

In [46]:
# Turn stat dictionaries into DataFrame
df = pd.DataFrame([effnetb2_stats, vit_stats])
df['model'] = ["EffNetB2", "ViT"]
df['test_acc'] = round (df['test_acc'] * 100, 2)
df

,test_loss,test_acc,number_of_paramters,model_size (MB),time_per_pred_cpu,model
0,0.281261,96.88,7705221,29.892145,0.0279,EffNetB2
1,0.063966,100.00,85800963,327.381945,0.0158,ViT


In [47]:
# compare Vit to EffNetB2
pd.DataFrame(data= (df.set_index('model').loc['ViT']/df.set_index('model').loc['EffNetB2']),
                        columns = ['Fit to EffNetB2 ratios']).T

,test_loss,test_acc,number_of_paramters,model_size (MB),time_per_pred_cpu
Fit to EffNetB2 ratios,0.227428,1.032205,11.135432,10.952106,0.566308


### 6.1 Visulaizeing speed vs performance

<function matplotlib.pyplot.show(close=None, block=None)>

In [48]:
fig, ax = plt.subplots(figsize= (12, 8))
scatter = ax.scatter(data= df,
                     x = 'time_per_pred_cpu',
                     y = 'test_acc',
                     c = ['blue', 'orange'],
                     s = 'model_size (MB)')
ax.set_title ("FoodVision Mini Inference Speed vs Performance", fontsize = 18)
ax.set_xlabel("prediction tie per image (second)", fontsize = 14)
ax.set_ylabel("Test accuracy (%)", fontsize = 14)
ax.tick_params (axis = 'both', labelsize = 12)

ax.grid (True)

# 3. annotate
for index, row in df.iterrows():
    ax.annotate(text = row['model'],
                xy = (row['time_per_pred_cpu']+0.0006, row ['test_acc']+0.03))
    
#  4. Create legedn
handles, labels = scatter.legend_elements (prop='sizes', alpha = 0.5)
models_size_legend = ax.legend(handles,
                               labels,
                               loc = 'lower right',
                               title = "Model size (MB)",
                               fontsize = 12)

plt.show

## 7. Food Vision minto on Gradio

In [49]:
# 

In [50]:
# import/install gradio
try: 
    import gradio as gr  
except:
    !pip -q install gradio
    import gradio as gr

print (f'Gradio version {gr.__version__}')

Gradio version 5.9.1


In [51]:
# put model to cput
effnetb2.to('cpu')
next(iter(effnetb2.parameters())).device

device(type='cpu')

In [52]:
# recreate workflow
def predict(img) ->Tuple [Dict, float]:

    start_time = timer()

    img = effnetb2_transforms(img).unsqueeze(0)

    effnetb2.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(effnetb2(img), dim = 1)

    pred_labels_probs = {class_names[i]:float (pred_probs[0][i]) for i in range (len(class_names))}

    pred_time = round (timer() - start_time , 5)

    return pred_labels_probs, pred_time


In [53]:
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))

random_image_path = random.sample (test_data_paths, k = 1)[0]

image = Image.open(random_image_path)

print (f'[INFO] Prediction on image at path: {random_image_path}\n')
pred_dict, pred_time = predict (img = image)
print (f'label and prbob:\n{pred_dict}')
print (f'Prediction Time : {pred_time} seconds')

[INFO] Prediction on image at path: ..\data\pizza_steak_sushi_20_percent\test\sushi\2903125.jpg

label and prbob:
{'pizza': 0.34204351902008057, 'steak': 0.3258858323097229, 'sushi': 0.33207061886787415}
Prediction Time : 0.09288 seconds


### 7.3 Creating example list

In [54]:
example_list = [[str(filepath)] for filepath in random.sample (test_data_paths, k = 3)]
example_list

[['..\\data\\pizza_steak_sushi_20_percent\\test\\steak\\108310.jpg'],
 ['..\\data\\pizza_steak_sushi_20_percent\\test\\pizza\\1067986.jpg'],
 ['..\\data\\pizza_steak_sushi_20_percent\\test\\sushi\\3886015.jpg']]

### 7.4 Build a Gradiao interface

In [55]:
title = "Food vision Mini 🍕🥩🍣"
description = "an EfficentNetB2 feature extract"
article = "Created at Hassan 09"

demo = gr.Interface(fn=predict,
                    inputs = gr.Image(type='pil'),
                    outputs = [gr.Label(num_top_classes = 3, label = "Prdictions"),
                                gr.Number(label = "prediction time")],
                    examples = example_list,
                    title = title,
                    description=description,
                    article=article)

demo.launch(debug=False,
           share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


## 8. Put FoodVision Mini on Gradio into a deployable app

### Creating a demo folder 

In [56]:
# creae good vision demo path
food_mini_demo_path = Path('demos/foodvision_mini/')

if food_mini_demo_path.exists():
    # shutil.rmtree(food_mini_demo_path)
    # food_mini_demo_path.mkdir(parents=True,
    #                           exist_ok=True)
    pass
    
else:
        food_mini_demo_path.mkdir(parents=True,
                                  exist_ok=True)
        
!ls demo/foodvision_min/
# print(os.listdir(food_mini_demo_path))

'ls' is not recognized as an internal or external command,
operable program or batch file.


### 8.4 Creating voder for exampels


In [57]:
foodvision_mini_examples_path =food_mini_demo_path / 'examples'

foodvision_mini_examples_path.mkdir(parents=True, exist_ok=True)

# E:\Dropbox\GithubRepo\Udemy\pytorch-deep-learning-main\pytorch-deep-learning-main - Copy\data\pizza_steak_sushi_20_percent\test
# 
# file_list =  '/pizza/' + '2582289.jpg', '/steak/' + '3622237.jpg', '/steak/' + '2582289.jpg')
# foodvision_mini_examples_path = ;.join ()
foodvision_mini_examples = [Path(test_dir / 'pizza' / '44810.jpg' ),
                                 Path(test_dir / 'sushi' / '46797.jpg'),
                                 Path(test_dir / 'steak' / '39461.jpg')]

# pizza_example = test_dir /'pizza/' + '2582289.jpg')
# steak_example = file('steak/' + '3622237.jpg')
# sushi_exampel = file('sushi' + '592799.jpg')


# foodvision_mini_examples_path = [test_dir / file('pizza/2582289.jpg') ,
#                                  test_dir / file('steak/3622237.jpg'),
#                                  test_dir / file('sushi/592799.jpg')]
for example in foodvision_mini_examples:
        destination = foodvision_mini_examples_path / example.name
        print (f'[INFO] Copying {example} to {destination}')
        shutil.copy2(src=example, dst=destination)



[INFO] Copying ..\data\pizza_steak_sushi_20_percent\test\pizza\44810.jpg to demos\foodvision_mini\examples\44810.jpg
[INFO] Copying ..\data\pizza_steak_sushi_20_percent\test\sushi\46797.jpg to demos\foodvision_mini\examples\46797.jpg
[INFO] Copying ..\data\pizza_steak_sushi_20_percent\test\steak\39461.jpg to demos\foodvision_mini\examples\39461.jpg


### 8.5 moving EffnetB2 to FoodVision Mini demo

In [58]:
# soure path
effnetb2_foodvision_mini_model_path_new = f"models/09_pre_trained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent_{10}_epochs.pth"

effnetb2_foodvision_mini_model_destination = food_mini_demo_path / effnetb2_foodvision_mini_model_path_new.split("/")[1]

try:
    print (f'[INFO] attempting to move {effnetb2_foodvision_mini_model_path} to  {effnetb2_foodvision_mini_model_destination}') 

    # move model
    shutil.move(src= effnetb2_foodvision_mini_model_path_new, dst = effnetb2_foodvision_mini_model_destination)

    print (f'[INfO] model move complete')

except:
        print (f'[INFO] move failed')


[INFO] move failed


### 8.6 Turn EffnetB2 model a python script

In [7]:
%%writefile demos/foodvision_mini/model.py
# function to create effnetb2
import torch
import torchvision

from torch import nn

def create_effnetb2_model (output_feature_count:int=3, drop_out = 0.3, seed:int=42)-> nn.Module:
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms= weights.transforms()
    # model = torchvision.models.efficientnet_b2(weights="DEFAULT").to(device)
    # model = torchvision.models.efficientnet_b2(weights=weights).to(device)
    model = torchvision.models.efficientnet_b2(weights=weights)


    # for param in model.features.parameters():
    for param in model.parameters():
                param.requires_grad = False

    # set_seeds()
    torch.manual_seed(seed)

    last_layer = list(model.children())[-1]
    last_child_layer = list(last_layer.children())[-1]
    last_input = last_child_layer.in_features

    model.classifier = nn.Sequential(
        nn.Dropout(p = drop_out, inplace=True),
        nn.Linear(in_features =last_input, 
                #   out_features = output_feature_count).to(device))
                  out_features = output_feature_count))                 

    # model.name = 'effnetb2'
    # print (f'[INFO] created new {model.name} model')
    
    return model, transforms

Overwriting demos/foodvision_mini/model.py


### 8.7 Turning our FoodVision Mini Gradio app into a Python script (`app.py`)

In [63]:
%%writefile demos/foodvision_mini/app.py
# 1. import and setup
import gradio as gr
import os
import torch

from model import create_effnetb2_model # put back in
from timeit import default_repeat as timer  
from typing import Tuple, Dict


# 2. Models and Transforms
effnetb2, effnetb2_transforms = create_effnetb2_model(output_feature_count=3)

effnetb2.load_state_dict(
    torch.load(
        # f=f"models/09_pre_trained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent_{10}_epochs.pth",
        f=f"09_pre_trained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent_{10}_epochs.pth",        
        map_location=torch.device('cpu')
    )
)

# 3. Predict Funciton
# def predict(img) -> Tuple[Dict, float]
# recreate workflow
def predict(img) ->Tuple [Dict, float]:
    from timeit import default_timer as timer
    start_time = timer()

    img = effnetb2_transforms(img).unsqueeze(0)

    effnetb2.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(effnetb2(img), dim = 1)

    pred_labels_probs = {class_names[i]:float (pred_probs[0][i]) for i in range (len(class_names))}

    pred_time = round (timer() - start_time , 5)

    return pred_labels_probs, pred_time

#  Gradio Ap
title = "Food vision Mini 🍕🥩🍣"
description = "an EfficentNetB2 feature extract"
article = "Created at Hassan 09"

# these will likely cause problems in relative paths. Need to fix. 
# example_list = [[str(filepath)] for filepath in random.sample (test_data_paths, k = 3)]
example_list = [['examples/' + example] for example in os.listdir('examples')]

demo = gr.Interface(fn=predict,
                    inputs = gr.Image(type='pil'),
                    outputs = [gr.Label(num_top_classes = 3, label = "Predictions"),
                                gr.Number(label = "prediction time")],
                    examples = example_list,
                    title = title,
                    description=description,
                    article=article)

# demo.launch(debug=False,
#            share=True)

Overwriting demos/foodvision_mini/app.py


In [61]:
%pwd
demo.launch(debug=False,
           share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


### 8.8 make requirments text

In [72]:
%%writefile demos/foodvision_mini/requirements.txt
torch
torchvision
gradio

Overwriting demos/foodvision_mini/requirements.txt


## 9. Deploying FoodVisoin Minto to huggingface spaces


In [145]:
# !dir demos\foodvision_mini

 Volume in drive E is TigerData
 Volume Serial Number is 4EEC-77CB

 Directory of e:\Dropbox\GithubRepo\Udemy\pytorch-deep-learning-main\pytorch-deep-learning-main - Copy\MyProjects\demos\foodvision_mini

01/08/2025  08:38 PM    <DIR>          .
01/08/2025  08:38 PM    <DIR>          ..
01/07/2025  07:51 PM        31,344,186 09_pre_trained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent_10_epochs.pth
09/21/2013  04:29 AM            63,129 2582289.jpg
09/21/2013  04:30 AM            71,470 3622237.jpg
09/21/2013  04:30 AM            53,561 39461.jpg
09/21/2013  04:29 AM            68,279 44810.jpg
09/21/2013  04:28 AM            76,013 46797.jpg
01/08/2025  08:24 PM             1,668 app.py
01/05/2025  04:11 PM    <DIR>          examples
01/07/2025  08:39 PM             1,026 model.py
01/08/2025  08:38 PM                28 requirements.txt
               9 File(s)     31,679,360 bytes
               3 Dir(s)  4,833,993,662,464 bytes free


In [2]:
import os
import zipfile

# Define source and destination
source_dir = 'demos/foodvision_mini'
zip_path = 'demos/foodvision_mini.zip'

# Create a ZIP file excluding unwanted patterns
def zip_excluding(source_dir, zip_path, exclude_patterns):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            # Exclude patterns for directories
            dirs[:] = [d for d in dirs if not any(ex in os.path.join(root, d) for ex in exclude_patterns)]
            # Exclude patterns for files
            for file in files:
                if not any(ex in os.path.join(root, file) for ex in exclude_patterns):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), source_dir))

exclude = ['.pyc', '.ipynb', '__pycache__', 'ipynb_checkpoints']
zip_excluding(source_dir, zip_path, exclude)

print(f"Zipped to {zip_path}")


Zipped to demos/foodvision_mini.zip


## 10. Creating FoodVision Big

We've spent the past few sections and chapters working on bringing FoodVision Mini to life.

And now we've seen it working in a live demo, how about we step things up a notch?

How?

FoodVision Big!

Since FoodVision Mini is trained on pizza, steak and sushi images from the [Food101 dataset](https://pytorch.org/vision/main/generated/torchvision.datasets.Food101.html) (101 classes of food x 1000 images each), how about we make FoodVision Big by training a model on all 101 classes!

We'll go from three classes to 101!

From pizza, steak, sushi to pizza, steak, sushi, hot dog, apple pie, carrot cake, chocolate cake, french fires, garlic bread, ramen, nachos, tacos and more!

How?

Well, we've got all the steps in place, all we have to do is alter our EffNetB2 model slightly as well as prepare a different dataset.

To finish Milestone Project 3, let's recreate a Gradio demo similar to FoodVision Mini (three classes) but for FoodVision Big (101 classes).

<img src="https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/09-model-deployment-foodvision-mini-to-foodvision-big.png" alt="foodvision mini model on three classes: pizza, steak, sushi and foodvision big on all of the 101 classes in the food101 dataset" width=900/>

*FoodVision Mini works with three food classes: pizza, steak and sushi. And FoodVision Big steps it up a notch to work across 101 food classes: all of the [classes in the Food101 dataset](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/food101_class_names.txt).*

### 10.1 Creating a model and transforms for FoodVision Big

When creating FoodVision Mini we saw that the EffNetB2 model was a good tradeoff between speed and performance (it performed well with a fast speed).

So we'll continue using the same model for FoodVision Big.

We can create an EffNetB2 feature extractor for Food101 by using our `create_effnetb2_model()` function we created above, in [section 3.1](https://www.learnpytorch.io/09_pytorch_model_deployment/#31-creating-a-function-to-make-an-effnetb2-feature-extractor), and passing it the parameter `num_classes=101` (since Food101 has 101 classes).

In [67]:
#  create effnetb2 model capable of fitting to 101 classes for Food101
effnetb2_food101, effnetb2_transforms = create_effnetb2_model(output_feature_count = 101)

[INFO] created new effnetb2 model


In [69]:
#  get summary
summary (effnetb2_food101,
         input_size=(1, 3, 224, 224),
         col_names = ["input_size", "output_size", "num_params", "trainable"],
         col_width = 20,
         row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 101]             --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

<img src="https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/09-effnetb2-feature-extractor-101-classes.png" width=900 alt="effnetb2 feature extractor with 100 output classes model summary"/>
                                                                                                                                                      
Nice!

See how just like our EffNetB2 model for FoodVision Mini the base layers are frozen (these are pretrained on ImageNet) and the outer layers (the `classifier` layers) are trainble with an ouput shape of `[batch_size, 101]` (`101` for 101 classes in Food101). 

Now since we're going to be dealing with a fair bit more data than usual, how about we add a little data augmentation to our transforms (`effnetb2_transforms`) to augment the training data.

> **Note:** Data augmentation is a technique used to alter the appearance of an input training sample (e.g. rotating an image or slightly skewing it) to artificially increase the diversity of a training dataset to hopefully prevent overfitting. You can see more on data augmentation in [04. PyTorch Custom Datasets section 6](https://www.learnpytorch.io/04_pytorch_custom_datasets/#6-other-forms-of-transforms-data-augmentation).

Let's compose a `torchvision.transforms` pipeline to use [`torchvision.transforms.TrivialAugmentWide()`](https://pytorch.org/vision/main/generated/torchvision.transforms.TrivialAugmentWide.html) (the same data augmentation used by the PyTorch team in their [computer vision recipes](https://pytorch.org/blog/how-to-train-state-of-the-art-models-using-torchvision-latest-primitives/#break-down-of-key-accuracy-improvements)) as well as the `effnetb2_transforms` to transform our training images.

In [70]:
# create Foo101 training data
food101_train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.TrivialAugmentWide(),
    effnetb2_transforms,
])

In [71]:
print(f'Traning transforms:\n{food101_train_transforms}\n')
print(f'Testing transforms:\n{effnetb2_transforms} ')

Traning transforms:
Compose(
    TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)
)

Testing transforms:
ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
) 


In [92]:
# set up data path
data_dir = Path('../data/food-101') # idfferent from 
# data_dir = '../data/food-101'
# use local for training
train_data = datasets.Food101 (root= data_dir,
                               split = 'train',
                               transform=food101_train_transforms,
                               download = False)

test_data = datasets.Food101(root=data_dir,
                            split= 'test',
                            transform=effnetb2_transforms,
                            download = False)



In [ ]:
# import tarfile

# file_path = "../data/food-101.tar.gz"
# extract_path = "../data/food-101"

# with tarfile.open(file_path, "r:gz") as tar:
#     tar.extractall(path=extract_path)

# print("Extraction complete!")


Extraction complete!
